## Import Dataset

In [1]:
!pip install scikit-learn pandas numpy torchview visualtorch gdown torch-geometric-temporal torch-cluster # then restart

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.3/102.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.4/997.4 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [16]:
!gdown --folder https://drive.google.com/drive/folders/1dydbU9HlSIgGQBzYMLogDNI27uO6wga7?usp=sharing

Retrieving folder contents
Retrieving folder 1KjgtXYS3NaIN6tik_daJWOl63IgzGso7 Data_สถานีชาร์จ
Processing file 1KMDUW5SuAI6kFWNPkfWbwDgcTSSvYxT8 รายงานสรุป-Demand-รายวัน-สถานีชาร์จ-01-2024.xlsx
Processing file 1hdn9FIx3p2tRGlTKHJDcPidkSPPtXoOx รายงานสรุป-Demand-รายวัน-สถานีชาร์จ-02-2024.xlsx
Processing file 10XGBjCursyMvv8EBxKO3i_2OOP9aRHP2 รายงานสรุป-Demand-รายวัน-สถานีชาร์จ-03-2024.xlsx
Processing file 1D09dkdAj-a_bCZixA2Fl2XS3GFWrllJD รายงานสรุป-Demand-รายวัน-สถานีชาร์จ-04-2024.xlsx
Processing file 17yXlMWcwocV7LYA5ZqI7Mfbt9U2pQHS- รายงานสรุป-Demand-รายวัน-สถานีชาร์จ-05-2024.xlsx
Processing file 1pNPnr9FucqI_j3YLWLyoLNnvvbzDQ3Rp รายงานสรุป-Demand-รายวัน-สถานีชาร์จ-06-2024.xlsx
Processing file 1ZqvUMcV1KusK_3P5cgODBvPEuFtX3CRK รายงานสรุป-Demand-รายวัน-สถานีชาร์จ-07-2024.xlsx
Processing file 1ApbincXLdZCwYAk4VWnZf9gV6cjUMmlW รายงานสรุป-Demand-รายวัน-สถานีชาร์จ-08-2024.xlsx
Processing file 1-r7wosGYXQZhRPuU9iMyI-eK41u1vEjT รายงานสรุป-Demand-รายวัน-สถานีชาร์จ-09-2024.xlsx
Processing fil

## Load & Clean the Data

In [3]:
import os
import re
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

# --------- CONFIGURATION ---------
ROOT_XLSX_DIR = "/kaggle/input/chula-data/Load-data"
CLEANED_CSV_DIR = "cleaned_data"
PREPROCESSED_CSV_DIR = "preprocessed_data"
FINAL_WIDE_CSV = "all_data_df.csv"
FINAL_LONG_CSV = "all_data_timeseries.csv"
# ---------------------------------

def clean_header_and_drop_unused_rows(tmp_df):
    tmp_df.columns = tmp_df.iloc[0]
    tmp_df = tmp_df[1:].reset_index(drop=True)
    if 'Date' in tmp_df.columns:
        tmp_df = tmp_df[~pd.isna(tmp_df['Date'])]
    return tmp_df

def process_excel_file(file_info):
    file_path, rel_path = file_info
    try:
        tmp_df = pd.read_excel(file_path)
        cleaned_df = clean_header_and_drop_unused_rows(tmp_df)
        output_path = os.path.join(CLEANED_CSV_DIR, rel_path).replace(".xlsx", ".csv")
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        cleaned_df.to_csv(output_path, index=False)
        return f"✅ Excel Processed: {file_path}"
    except Exception as e:
        return f"❌ Excel Error in {file_path}: {str(e)}"

def preprocess_and_add_datetime(tmp_df, filename):
    match = re.search(r"(\d{2})-(\d{4})", filename)
    if not match:
        raise ValueError(f"❌ Cannot extract date from filename: {filename}")

    start_month = int(match.group(1))
    start_year = int(match.group(2))
    tmp_df = tmp_df.reset_index(drop=True)

    date_range = pd.date_range(start=datetime(start_year, start_month, 1), periods=len(tmp_df), freq='D')
    tmp_df['Date'] = date_range

    time_cols = [col for col in tmp_df.columns if col != 'Date']
    tmp_df[time_cols] = tmp_df[time_cols].apply(pd.to_numeric, errors='coerce')
    return tmp_df

def process_csv_file(file_info):
    file_path, rel_path = file_info
    try:
        tmp_df = pd.read_csv(file_path)
        processed_df = preprocess_and_add_datetime(tmp_df, os.path.basename(file_path))

        station_name = os.path.normpath(rel_path).split(os.sep)[0]
        processed_df.insert(0, 'station_name', station_name)

        output_path = os.path.join(PREPROCESSED_CSV_DIR, rel_path)
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        processed_df.to_csv(output_path, index=False)
        return f"✅ CSV Processed: {file_path}"
    except Exception as e:
        return f"❌ CSV Error in {file_path}: {str(e)}"

def gather_files(root_dir, extension):
    files = []
    for subdir, _, filenames in os.walk(root_dir):
        for f in filenames:
            if f.endswith(extension):
                full = os.path.join(subdir, f)
                rel = os.path.relpath(full, root_dir)
                files.append((full, rel))
    return files

def concatenate_preprocessed_data(output_dir):
    all_data = []
    for subdir, _, files in os.walk(output_dir):
        for file in files:
            if file.endswith(".csv"):
                try:
                    df = pd.read_csv(os.path.join(subdir, file))
                    all_data.append(df)
                except Exception as e:
                    print(f"❌ Failed to read {file}: {e}")
    return pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()

def convert_to_timeseries_long_format(df):
    time_columns = [col for col in df.columns if re.match(r"^\d{1,2}:\d{2}$", str(col))]
    long_df = df.melt(id_vars=['station_name', 'Date'], value_vars=time_columns,
                      var_name='Time', value_name='Electricity(kW)')
    long_df['Date'] = pd.to_datetime(long_df['Date'].astype(str) + ' ' + long_df['Time'])
    long_df.drop(columns=['Time'], inplace=True)
    long_df.sort_values(by=['station_name', 'Date'], inplace=True)
    return long_df

# ----------- MAIN EXECUTION FLOW -----------
if __name__ == "__main__":
    # Step 1: Clean Excel files to CSV
    xlsx_files = gather_files(ROOT_XLSX_DIR, ".xlsx")
    with Pool(cpu_count()) as pool:
        results = list(tqdm(pool.imap_unordered(process_excel_file, xlsx_files), total=len(xlsx_files)))
    for res in results:
        print(res)

    # Step 2: Preprocess cleaned CSVs
    csv_files = gather_files(CLEANED_CSV_DIR, ".csv")
    with Pool(cpu_count()) as pool:
        results = list(tqdm(pool.imap_unordered(process_csv_file, csv_files), total=len(csv_files)))
    for res in results:
        print(res)

    # Step 3: Concatenate all preprocessed CSVs
    all_data_df = concatenate_preprocessed_data(PREPROCESSED_CSV_DIR)
    if not all_data_df.empty:
        all_data_df.to_csv(FINAL_WIDE_CSV, index=False)
        print(f"✅ Wide-format saved to {FINAL_WIDE_CSV}")

        # Step 4: Convert to long time series format
        long_df = convert_to_timeseries_long_format(all_data_df)
        long_df.to_csv(FINAL_LONG_CSV, index=False)
        print(f"✅ Long-format saved to {FINAL_LONG_CSV}")
    else:
        print("⚠️ No data found for concatenation.")

100%|██████████| 71/71 [00:02<00:00, 30.31it/s]

✅ Excel Processed: /kaggle/input/chula-data/Load-data/Data_สถานีชาร์จ/รายงานสรุป-Demand-รายวัน-สถานีชาร์จ-02-2024.xlsx
✅ Excel Processed: /kaggle/input/chula-data/Load-data/Data_สถานีชาร์จ/รายงานสรุป-Demand-รายวัน-สถานีชาร์จ-06-2024.xlsx
✅ Excel Processed: /kaggle/input/chula-data/Load-data/Data_สถานีชาร์จ/รายงานสรุป-Demand-รายวัน-สถานีชาร์จ-09-2024.xlsx
✅ Excel Processed: /kaggle/input/chula-data/Load-data/Data_สถานีชาร์จ/รายงานสรุป-Demand-รายวัน-สถานีชาร์จ-03-2024.xlsx
✅ Excel Processed: /kaggle/input/chula-data/Load-data/Data_สถานีชาร์จ/รายงานสรุป-Demand-รายวัน-สถานีชาร์จ-11-2024.xlsx
✅ Excel Processed: /kaggle/input/chula-data/Load-data/Data_สถานีชาร์จ/รายงานสรุป-Demand-รายวัน-สถานีชาร์จ-04-2024.xlsx
✅ Excel Processed: /kaggle/input/chula-data/Load-data/Data_สถานีชาร์จ/รายงานสรุป-Demand-รายวัน-สถานีชาร์จ-07-2024.xlsx
✅ Excel Processed: /kaggle/input/chula-data/Load-data/Data_สถานีชาร์จ/รายงานสรุป-Demand-รายวัน-สถานีชาร์จ-05-2024.xlsx
✅ Excel Processed: /kaggle/input/chula-data/Load


100%|██████████| 71/71 [00:00<00:00, 96.37it/s]


✅ CSV Processed: cleaned_data/Data_อาคารจามจุรี 9/รายงานสรุป-Demand-รายวัน-อาคารจามจุรี9-11-2024.csv
✅ CSV Processed: cleaned_data/Data_อาคารจามจุรี 9/รายงานสรุป-Demand-รายวัน-อาคารจามจุรี9-02-2024.csv
✅ CSV Processed: cleaned_data/Data_อาคารจามจุรี 9/รายงานสรุป-Demand-รายวัน-อาคารจามจุรี9-09-2024.csv
✅ CSV Processed: cleaned_data/Data_อาคารจามจุรี 9/รายงานสรุป-Demand-รายวัน-อาคารจามจุรี9-12-2023.csv
✅ CSV Processed: cleaned_data/Data_อาคารจามจุรี 9/รายงานสรุป-Demand-รายวัน-อาคารจามจุรี9-07-2024.csv
✅ CSV Processed: cleaned_data/Data_อาคารจามจุรี 9/รายงานสรุป-Demand-รายวัน-อาคารจามจุรี9-08-2024.csv
✅ CSV Processed: cleaned_data/Data_อาคารจามจุรี 9/รายงานสรุป-Demand-รายวัน-อาคารจามจุรี9-05-2024.csv
✅ CSV Processed: cleaned_data/Data_อาคารจามจุรี 9/รายงานสรุป-Demand-รายวัน-อาคารจามจุรี9-03-2024.csv
✅ CSV Processed: cleaned_data/Data_อาคารจามจุรี 9/รายงานสรุป-Demand-รายวัน-อาคารจามจุรี9-04-2024.csv
✅ CSV Processed: cleaned_data/Data_อาคารจามจุรี 9/รายงานสรุป-Demand-รายวัน-อาคารจามจุรี9-01

## Define Weight

In [5]:
import pandas as pd

# Create the DataFrame
station_weights_df = pd.DataFrame({
    "station_name": [
        "Data_สถานีชาร์จ",
        "Data_อาคารจามจุรี 9",
        "Data_อาคารวิทยนิเวศน์",
        "Data_อาคารจุลจักรพงษ์",
        "Data_อาคารบรมราชกุมารี",
        "Data_อาคารจามจุรี4",
    ],
    "normalized_reverse_weight": [
        1.000000,
        1.000000,
        1.000000,        1.002786,
        1.002786,
        1.094225,
    ]
})

## Experiment [Clean Data]

In [6]:
def preprocess(long_df):
    long_df.loc[long_df['Electricity(kW)'] < 0, 'Electricity(kW)'] = 0
    return long_df
# long_df_tmp = preprocess(long_df_new)
long_df = preprocess(long_df)

/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)


In [7]:
long_df['station_name'].unique()

array(['Data_สถานีชาร์จ', 'Data_อาคารจามจุรี 9', 'Data_อาคารจามจุรี4',
       'Data_อาคารจุลจักรพงษ์', 'Data_อาคารบรมราชกุมารี',
       'Data_อาคารวิทยนิเวศน์'], dtype=object)

## Split train,valid and test

In [8]:
def split_train_test_data(long_df,long_df_new):
    # Define ratios
    train_ratio = 0.8
    test_ratio = 0.2  # Optional, just for clarity (1 - train_ratio)
    
    # Create empty lists to collect per-station splits
    train_list = []
    test_list = []
    
    # Split per station
    for station, station_df in long_df_new.groupby('station_name'):
        station_df = station_df.sort_values('Date')
        n = len(station_df)
    
        train_end = int(n * train_ratio)
    
        train_list.append(station_df.iloc[:train_end])
        test_list.append(station_df.iloc[train_end:])
    
    # Combine all stations back into global sets
    train_df = pd.concat(train_list).reset_index(drop=True)
    # Create empty lists to collect per-station splits
    train_list = []
    test_list = []
    for station, station_df in long_df.groupby('station_name'):
        station_df = station_df.sort_values('Date')
        n = len(station_df)
    
        train_end = int(n * train_ratio)
    
        train_list.append(station_df.iloc[:train_end])
        test_list.append(station_df.iloc[train_end:])
    
    test_df_new = pd.concat(test_list).reset_index(drop=True)
    
    return train_df,test_df_new
train_df,test_df = split_train_test_data(long_df,long_df)

In [9]:
locations = {
    "Data_สถานีชาร์จ": (13.73624, 100.52995), #Station_name, latitude,longitude
    "Data_อาคารจามจุรี4": (13.73260, 100.53177),
    "Data_อาคารจามจุรี 9": (13.73380, 100.53045),
    "Data_อาคารจุลจักรพงษ์": (13.73684, 100.52852),
    "Data_อาคารบรมราชกุมารี": (13.73800, 100.52905),
    "Data_อาคารวิทยนิเวศน์": (13.73723, 100.53015),
}


## Create Graph

In [22]:
train_df['station_name'].unique()

array(['Data_สถานีชาร์จ', 'Data_อาคารจามจุรี 9', 'Data_อาคารจามจุรี4',
       'Data_อาคารจุลจักรพงษ์', 'Data_อาคารบรมราชกุมารี',
       'Data_อาคารวิทยนิเวศน์'], dtype=object)

In [23]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal import ASTGCN
from torch_geometric.utils import dense_to_sparse
from torch.utils.data import DataLoader, Dataset
from torch.amp import GradScaler, autocast
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import torch.nn as nn

# 1. Prepare device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. Graph setup
locations = {
    "Data_สถานีชาร์จ": (13.73624, 100.52995),
    "Data_อาคารจามจุรี4": (13.73260, 100.53177),
    "Data_อาคารจามจุรี 9": (13.73380, 100.53045),
    "Data_อาคารจุลจักรพงษ์": (13.73684, 100.52852),
    "Data_อาคารบรมราชกุมารี": (13.73800, 100.52905),
    "Data_อาคารวิทยนิเวศน์": (13.73723, 100.53015),
}
station_names = list(locations.keys())
num_nodes = len(station_names)

# fully connected edges (i != j)
edge_index = torch.tensor(
    [[i, j] for i in range(num_nodes) for j in range(num_nodes) if i != j],
    dtype=torch.long,
).t().contiguous().to(device)

# 3. Pivot helper
def pivot_to_tensor(df, seq_len):
    df_pv = df.pivot(index='Date', columns='station_name', values='Electricity(kW)')
    df_pv = df_pv[station_names].fillna(0.)
    windows = []
    for i in range(len(df_pv) - seq_len + 1):
        win = df_pv.iloc[i:i+seq_len].values  # (seq_len, N)
        windows.append(win.T)                 # (N, seq_len)
    arr = np.stack(windows, axis=0)          # (T, N, seq_len)
    return torch.tensor(arr, dtype=torch.float)

# 4. Prepare data
len_input = 96
pred_len  = 96

X = pivot_to_tensor(train_df, len_input + pred_len)
X_in  = X[:, :, :len_input]
X_out = X[:, :, len_input:]

class TemporalDataset(Dataset):
    def __init__(self, X_i, X_o):
        self.X_i, self.X_o = X_i, X_o
    def __len__(self):
        return len(self.X_i)
    def __getitem__(self, idx):
        return self.X_i[idx], self.X_o[idx]

loader = DataLoader(TemporalDataset(X_in, X_out), batch_size=512, shuffle=True)

# 5. Model definition
class ASTGCN_V2(nn.Module):
    def __init__(self, num_nodes, **kwargs):
        super().__init__()
        self.astgcn = ASTGCN(**kwargs)
        self.node_emb1 = nn.Parameter(torch.randn(num_nodes, 10))
        self.node_emb2 = nn.Parameter(torch.randn(10, num_nodes))

    def forward(self, x, edge_index=None):
        A_int = F.relu(self.node_emb1 @ self.node_emb2)  # (N, N)
        A_adp = F.softmax(A_int, dim=1)
        ei_adp, _ = dense_to_sparse(A_adp)
        out = self.astgcn(x, ei_adp.to(x.device))
        return F.relu(out)

config = {
    "nb_block": 2,
    "in_channels": 1,
    "K": 2,
    "nb_chev_filter": 64,
    "nb_time_filter": 64,
    "time_strides": 1,
    "num_for_predict": pred_len,
    "len_input": len_input,
    "num_of_vertices": num_nodes,
    "normalization": "sym",
    "bias": True,
}
max_lr = 1e-2
model     = ASTGCN_V2(num_nodes=num_nodes, **config).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=max_lr, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=max_lr,
    steps_per_epoch=len(loader),
    epochs=5,
    pct_start=0.3,
)
scaler = GradScaler()

# 6. Training
model.train()
for epoch in range(5):
    total_loss = 0.0
    for X_batch, Y_batch in tqdm(loader, desc=f"Epoch {epoch+1:02d}"):
        Xb = X_batch.unsqueeze(2).to(device)  # [B, N, 1, len_input]
        Yb = Y_batch.to(device)

        optimizer.zero_grad()
        with autocast(device_type="cuda"):  # now uses torch.amp.autocast
            preds = model(Xb, edge_index)
            loss  = criterion(preds, Yb)

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        scaler.step(optimizer)  # this performs optimizer.step()
        scaler.update()
        scheduler.step()        # now correctly after optimizer.step()


        total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    print(f"Epoch {epoch+1:02d} — Avg Loss: {avg_loss:.4f}")

Using device: cuda


Epoch 01:   0%|          | 0/55 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch 01 — Avg Loss: 21378.1381


Epoch 02:   0%|          | 0/55 [00:00<?, ?it/s]

...

Epoch 03 — Avg Loss: 21109.9218


Epoch 04:   0%|          | 0/55 [00:00<?, ?it/s]

Epoch 04 — Avg Loss: 21036.1655


Epoch 05:   0%|          | 0/55 [00:00<?, ?it/s]

Epoch 05 — Avg Loss: 21007.6627


In [ ]:
class ASTGCNWrapper(nn.Module):
    def __init__(self, model, edge_index):
        super().__init__()
        self.model = model
        self.edge_index = edge_index

    def forward(self, x):
        return self.model(x, self.edge_index)
from torchview import draw_graph

# Wrap the model with fixed edge_index
wrapped_model = ASTGCNWrapper(model, edge_index)

# Provide the correct input shape: (batch_size, num_nodes, 1, len_input)
draw_graph(
    wrapped_model,
    input_size=(1, num_nodes, 1, len_input),
    expand_nested=True,
    roll=True,
    show_shapes=True,
).visual_graph.render("astgcn_graph_v2.0", format="png")

## Evaluate

In [24]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Pivot and concatenate train+test
df_all = pd.concat([train_df, test_df], ignore_index=True)
pivot = (df_all
         .pivot(index='Date', columns='station_name', values='Electricity(kW)')
         .reindex(columns=station_names)      # ensure correct station order
         .fillna(0.0))
dates = pivot.index
T = len(dates)

# 2. Build every possible sliding window of length `len_input`
max_start = T - len_input - prediction_length + 1  # total windows
windows = []
for t0 in range(max_start):
    arr = pivot.iloc[t0:t0+len_input].values       # (len_input, N)
    windows.append(arr.T)                          # → (N, len_input)
X_all = np.stack(windows, axis=0)                  # (W, N, len_input)
X_all = torch.from_numpy(X_all).float().unsqueeze(2)  # (W, N, 1, len_input)

# 3. Batch through the model in eval mode
batch_size = 512
loader = DataLoader(TensorDataset(X_all), batch_size=batch_size, shuffle=False)

model.eval()
preds = []
with torch.no_grad():
    for (Xb,) in tqdm(loader,desc="batch"):
        Xb = Xb.to(device)
        yb = model(Xb, edge_index)               # → (B, N, prediction_length)
        preds.append(yb.cpu().numpy())
preds = np.concatenate(preds, axis=0)            # (W, N, pred_len)

# 4. Take only the *first-step* forecast (you can slice other horizons similarly)
first_step = preds[:, :, 0]                      # (W, N)

# 5. Build a long DataFrame of all predictions
#    window w predicts for date = dates[w + len_input]
pred_dates = dates[len_input : len_input + first_step.shape[0]]
records = []
for w, pd_dt in enumerate(pred_dates):
    for i, station in enumerate(station_names):
        records.append((pd_dt, station, first_step[w, i]))
df_preds = pd.DataFrame(records, columns=['Date','station_name','Predicted(kW)'])

# 6. Merge with test_df (this yields exactly len(test_df)=40 839 rows)
df_merged = (test_df
             .merge(df_preds, on=['Date','station_name'], how='left')
             .sort_values(['Date','station_name'])
             .reset_index(drop=True))

# after your merge:
df_eval = df_merged.dropna(subset=['Predicted(kW)']).copy()

# compute MAE only on the non‐NaN rows
mae = mean_absolute_error(
    df_eval['Electricity(kW)'].values,
    df_eval['Predicted(kW)'].values
)
print(f"Test MAE (first‐step, dropping {len(df_merged) - len(df_eval)} rows with no pred): {mae:.4f}")

batch:   0%|          | 0/68 [00:00<?, ?it/s]

Test MAE (first‐step, dropping 570 rows with no pred): 88.9843


In [25]:
import pandas as pd
import numpy as np

# Merge weights into evaluation DataFrame
df_eval = df_eval.merge(station_weights_df, on='station_name', how='left')

# Compute weighted absolute error
df_eval['abs_error'] = np.abs(df_eval['Electricity(kW)'] - df_eval['Predicted(kW)'])
df_eval['weighted_abs_error'] = df_eval['abs_error'] * df_eval['normalized_reverse_weight']

# Compute weighted actual value
df_eval['weighted_actual'] = df_eval['Electricity(kW)'] * df_eval['normalized_reverse_weight']

# Calculate WAPE
wape = df_eval['weighted_abs_error'].sum() / df_eval['weighted_actual'].sum()
print(f"WAPE (weighted): {wape:.4f} or {wape*100:.2f}%")

WAPE (weighted): 0.9659 or 96.59%


In [11]:
df_eval.to_csv("df_eval.csv",index=False)

In [ ]:
df_eval

## Optimize runtime & Inference

In [26]:
import torch

# assume `model` is your trained ASTGCN_V2 on CUDA or CPU
model.eval()
model.to("cpu")  # ONNX export is easiest on CPU

# example dummy input matching your train-time shape: [B, N, 1, len_input]
# here B=1 for tracing, N=num_nodes, len_input=96
dummy_input = torch.randn(1, num_nodes, 1, len_input, dtype=torch.float)

# export
torch.onnx.export(
    model,
    (dummy_input, edge_index.cpu()),             # model inputs
    "astgcn_v2.onnx",                             # output file
    export_params=True,                           # store weights
    opset_version=14,                             # recommended >= 12
    do_constant_folding=True,                     # fuse constants
    input_names=["input", "edge_index"],          
    output_names=["output"],
    dynamic_axes={
        "input": {0: "batch_size", 3: "seq_len"}, # batch & time dims dynamic
        "output": {0: "batch_size", 2: "pred_len"},
    }
)
print("ONNX model saved to astgcn_v2.onnx")

/usr/local/lib/python3.11/dist-packages/torch/onnx/symbolic_opset9.py:5331: UserWarning: Exporting aten::index operator with indices of type Byte. Only 1-D indices are supported. In any other case, this will produce an incorrect ONNX graph.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/onnx/symbolic_opset9.py:5383: UserWarning: Exporting aten::index operator of advanced indexing in opset 14 is achieved by combination of multiple ONNX operators, including Reshape, Transpose, Concat, and Gather. If indices include negative values, the exported graph will produce incorrect results.
  warnings.warn(


ONNX model saved to astgcn_v2.onnx


In [31]:
!pip install onnxruntime-tools onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 56.0 MB/s eta 0:00:00:00:0100:01


In [49]:
import torch
import onnx
from onnxruntime_tools import optimizer
from onnxruntime_tools.transformers.onnx_model_bert import BertOptimizationOptions

def export_to_onnx(
    model: torch.nn.Module,
    edge_index: torch.Tensor,
    output_path: str = "model.onnx",
    optimized_path: str = None,
    len_input: int = 96,
    device: str = "cpu",
    opset_version: int = 14,
    do_optimize: bool = True,
    model_type: str = "bert",
    num_heads: int = 1,
    num_nodes: int = 6,
):
    """
    Exports a PyTorch ASTGCN_V2 model (with dynamic batch & seq dims) to ONNX,
    then optionally runs ONNX Runtime optimizations.

    Args:
        model: the trained ASTGCN_V2 instance.
        edge_index: Tensor of shape [2, E] describing your graph.
        output_path: where to write the raw ONNX file.
        optimized_path: if provided and do_optimize=True, saves optimized model here.
                        if None, overwrites output_path.
        len_input: the sequence length used at export time.
        device: "cpu" or "cuda" — where to place the model/dummy for export.
        opset_version: ONNX opset to target.
        do_optimize: whether to run ONNX Runtime graph optimizations.
        model_type: passed to optimizer.optimize_model (default "bert").
        num_heads: dummy heads count for optimizer; not critical for ASTGCN.

    Returns:
        path to the final ONNX file (optimized_path or output_path).
    """
    # 1) Prepare model & dummy
    model.eval()
    model = model.to(device)
    dummy_input = torch.randn(1,num_nodes, 1, len_input, device=device)
    ei = edge_index.to(device)

    # 2) Export to ONNX
    torch.onnx.export(
        model,
        (dummy_input, ei),
        output_path,
        export_params=True,
        opset_version=opset_version,
        do_constant_folding=True,
        input_names=["input", "edge_index"],
        output_names=["output"],
        dynamic_axes={
            "input": {0: "batch_size", 3: "seq_len"},
            "output": {0: "batch_size", 2: "pred_len"},
        },
    )
    print(f"✅ Raw ONNX exported to {output_path}")

    final_path = output_path

    # 3) Optional optimization
    if do_optimize:
        optimized_path = optimized_path or output_path
        try:
            opt_opts = BertOptimizationOptions(model_type)
            opt_model = optimizer.optimize_model(
                output_path,
                model_type=model_type,
                num_heads=num_heads,
                optimization_options=opt_opts,
            )
            opt_model.save_model_to_file(optimized_path)
            final_path = optimized_path
            print(f"✅ Optimized ONNX saved to {optimized_path}")
        except Exception as e:
            print(f"⚠️ Optimization skipped: {e}")

    return final_path
# Export for CPU (default), with optimization:
onnx_path = export_to_onnx(
    model,
    edge_index,
    output_path="astgcn_cpu.onnx",
    optimized_path="astgcn_cpu_opt.onnx",
    len_input=96,
    device="cpu",
    do_optimize=True,
    num_nodes = num_nodes
)

# Export for GPU (with CUDAExecutionProvider support), without optimization:
onnx_path_gpu = export_to_onnx(
    model,
    edge_index,
    output_path="astgcn_gpu.onnx",
    optimized_path=None,
    len_input=96,
    device="cuda",
    do_optimize=False,
    num_nodes = num_nodes
)

✅ Raw ONNX exported to astgcn_cpu.onnx
⚠️ Optimization skipped: invalid literal for int() with base 10: 'unk__0'
✅ Raw ONNX exported to astgcn_gpu.onnx


In [66]:
import numpy as np
import torch
import onnxruntime as ort

class InferenceModel:
    def __init__(self, onnx_path="astgcn_v2.onnx", device="cpu"):
        providers = (["CUDAExecutionProvider","CPUExecutionProvider"]
                     if device.startswith("cuda") else ["CPUExecutionProvider"])
        self.sess = ort.InferenceSession(onnx_path, providers=providers)

        # Inspect the ONNX inputs
        inputs = self.sess.get_inputs()
        names = [inp.name for inp in inputs]
        if len(names) == 2:
            # graph expects [X, edge_index]
            self.input_name, self.edge_name = names
            self.need_edge = True
        elif len(names) == 1:
            # graph only expects [X], edge_index is built-in
            self.input_name = names[0]
            self.edge_name = None
            self.need_edge = False
        else:
            raise RuntimeError(f"Unexpected number of inputs in ONNX model: {len(names)}")

        self.output_name = self.sess.get_outputs()[0].name

    def forecast(self, X: torch.Tensor, edge_index: torch.Tensor = None) -> torch.Tensor:
        """
        X: [B, N, 1, seq_len]
        edge_index: [2, E] (only required if the ONNX session expects it)
        """
        Xn = X.cpu().numpy().astype(np.float32)
        feed = {self.input_name: Xn}

        if self.need_edge:
            if edge_index is None:
                raise ValueError("This model requires edge_index, but none was given.")
            En = edge_index.cpu().numpy().astype(np.int64)
            feed[self.edge_name] = En

        out = self.sess.run([self.output_name], feed)[0]
        return torch.from_numpy(out)

# --- Example usage ---

# Case 1: ONNX with two inputs
inf = InferenceModel("astgcn_cpu.onnx", device="cpu")
X_test = torch.randn(8, num_nodes, 1, len_input)
preds = inf.forecast(X_test, edge_index)                # must pass edge_index
preds

tensor([[[12799.7539,     0.0000,  5685.4536,  ..., 14815.2021,
          15516.9492, 12834.8281],
         [12846.3926,     0.0000,  5713.2607,  ..., 14879.3564,
          15589.9453, 12883.4619],
         [12809.1348,     0.0000,  5696.7539,  ..., 14831.0449,
          15529.9922, 12845.7881],
         [12799.8408,     0.0000,  5685.4819,  ..., 14815.3086,
          15517.1387, 12834.9121],
         [12794.8584,     0.0000,  5685.0708,  ..., 14810.7051,
          15509.5195, 12830.2715],
         [12799.6885,     0.0000,  5685.4453,  ..., 14815.1416,
          15516.8145, 12834.7617]],

        [[12799.7783,     0.0000,  5685.4624,  ..., 14815.2324,
          15516.9619, 12834.8525],
         [12846.3350,     0.0000,  5713.2412,  ..., 14879.2998,
          15589.8691, 12883.4111],
         [12809.0234,     0.0000,  5696.7124,  ..., 14830.9219,
          15529.8594, 12845.6777],
         [12799.0996,     0.0000,  5685.1938,  ..., 14814.4795,
          15516.2998, 12834.1787],
        

In [58]:
len_input

96

In [76]:
%%time
X_test = torch.randn(1, num_nodes, 1, len_input)
preds = inf.forecast(X_test, edge_index)                # must pass edge_index
# preds

CPU times: user 71.4 ms, sys: 0 ns, total: 71.4 ms
Wall time: 35.8 ms


In [ ]:
import time
import torch
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# (reuse your existing InferenceModel instance `inf` and `edge_index`, plus num_nodes, len_input)

process = psutil.Process()
batch_sizes = [10**i for i in range(11)]  # [2, 4, 8, …, 1024]

records = []
for b in batch_sizes:
    X_test = torch.randn(b, num_nodes, 1, len_input)

    # Reset peak memory if on GPU
    use_cuda = torch.cuda.is_available() and inf.sess.get_providers()[0].startswith("CUDA")
    if use_cuda:
        torch.cuda.reset_peak_memory_stats()

    # Memory before
    mem_before = torch.cuda.max_memory_allocated() if use_cuda else process.memory_info().rss

    # Time the inference
    start = time.perf_counter()
    _ = inf.forecast(X_test, edge_index if inf.need_edge else None)
    if use_cuda:
        torch.cuda.synchronize()
    end = time.perf_counter()

    # Memory after / peak
    mem_after = torch.cuda.max_memory_allocated() if use_cuda else process.memory_info().rss

    records.append({
        "batch_size": b,
        "time_ms": (end - start) * 1000,
        "mem_mb": (mem_after - mem_before) / (1024**2),
    })

# Build DataFrame
df = pd.DataFrame(records)

# Plot latency
plt.figure()
plt.plot(df["batch_size"], df["time_ms"], marker='o')
plt.xscale('log', base=2)
plt.xlabel("Batch Size (log2 scale)")
plt.ylabel("Inference Time (ms)")
plt.title("Latency vs Batch Size")
plt.tight_layout()
plt.show()

# Plot memory usage
plt.figure()
plt.plot(df["batch_size"], df["mem_mb"], marker='o')
plt.xscale('log', base=2)
plt.xlabel("Batch Size (log2 scale)")
plt.ylabel("Δ Memory (MB)")
plt.title("Memory Δ vs Batch Size")
plt.tight_layout()
plt.show()

In [62]:
preds.shape

torch.Size([8, 6, 96])